In [1]:
from glob import glob
import pickle

import hatchet as ht
import numpy as np
import pandas as pd

import thicket as th

DATA_DIR = "../data/parallel-sorting"

# 1. Read files into Thicket

- `glob`

In [2]:
data = glob(f"{DATA_DIR}/**/*.cali", recursive=True)
print(f"Total files: {len(data)}")

# Read caliper files without filling the profile index as it expensive and unnecessary in our case
tk = th.Thicket.from_caliperreader(
    data,
    fill_perfdata=False
)
print(f"DataFrame shape {tk.dataframe.shape}")
print(f"Metadata shape: {tk.metadata.shape}")

Total files: 15734


(2/2) Creating Thicket: 100%|██████████| 15733/15733 [02:33<00:00, 102.39it/s]

DataFrame shape (157459, 16)
Metadata shape: (15732, 65)


# 2. Modify and Filter Metadata Values

In [3]:
META_FIX_DICT = {
    "Algorithm": {
        "Quicksort": "QuickSort",
        "bitonic_sort": "BitonicSort",
        "merge_sort": "MergeSort",
        "Merge Sort": "MergeSort",
        "quicksort": "QuickSort",
        "odd_even_sort": "OddEvenSort",
        "Merge sort": "MergeSort",
        "Quick Sort": "QuickSort",
        "Sample Sort": "SampleSort",
        "Bitonic_Sort": "BitonicSort",
        "Merge_Sort": "MergeSort",
        "Quick_Sort": "QuickSort",
        "OddEvenTranspositionSort": "OddEvenSort",
        "Bitonic Sort": "BitonicSort",
        "Selection Sort": "SelectionSort",
        "Bucketsort": "BucketSort",
        "selection_sort": "SelectionSort",
        "Mergesort": "MergeSort",
        "mergesort": "MergeSort",
        "oddEven": "OddEvenSort",
        "Quick(Sample) Sort": "QuickSort",
        "Odd Even Transposition Sort": "OddEvenSort",
        "RadixSort Sort": "RadixSort",
        "Bucket Sort": "BucketSort",
        "Odd Even Sort": "OddEvenSort",
        "Odd-Even Sort": "OddEvenSort",
        "Countsort": "CountSort",
        "OddevenSort": "OddEvenSort",
        "oddeven_sort": "OddEvenSort",
        "bucket": "BucketSort",
        "Radix Sort": "RadixSort",
        "Odd-Even Bubble Sort": "OddEvenSort",
        "Bubble_Sort": "OddEvenSort",
        "Bubblesort": "OddEvenSort",
        "Bubble Sort(Odd/Even)": "OddEvenSort",
        "Bubble/Odd-Even Sort": "OddEvenSort",
        "Parallel Bubble Sort": "OddEvenSort",
        "BubbleSort": "OddEvenSort",
        "Radix": "RadixSort",
        "Bitonic": "BitonicSort",
    },
    "InputType": {
        "perturbed_array": "1%perturbed",
        "sorted_array": "Sorted",
        "random_array": "Random",
        "ascending_array": "Sorted",
        "descending_array": "Reverse",
        "reversed_array": "Reverse",
        "reversedSort": "Reverse",
        "1% Perturbed": "1%perturbed",
        "reverse_sorted": "Reverse",
        "1perturbed": "1%perturbed",
        r"1%%perturbed": "1%perturbed",
        "1 Perturbed": "1%perturbed",
        "1 perturbed": "1%perturbed",
        "Reverse Sorted": "Reverse",
        "1%Perturbed": "1%perturbed",
        "1% perturbation": "1%perturbed",
        "1percentperturbed": "1%perturbed",
        "1 percent noise": "1%perturbed",
        "reverse sorted": "Reverse",
        "sorted_1%_perturbed": "1%perturbed",
        "Reversesorted": "Reverse",
        "ReverseSorted": "Reverse",
        "Reverse_Sorted": "Reverse",
        "ReversedSort": "Reverse",
        "Sorted_1%_perturbed": "1%perturbed",
        "Randomized": "Random",
        "Reversed": "Reverse",
        "reversed": "Reverse",
        "sorted": "Sorted",
        "random": "Random",
        "nearly": "Nearly",
        "reverse": "Reverse",
        " Reverse sorted": "Reverse",
        "Perturbed": "1%perturbed",
        "perturbed": "1%perturbed",
    },
    "Datatype": {
        "integer": "int",
        "Int": "int",
        "Integer": "int",
        "Double": "double",
    },
}

META_WHITELIST_DICT = {
    "InputType": ["Random", "Sorted", "Reverse", "1%perturbed", "Nearly"],
    "Algorithm": [
        "BitonicSort",
        "BucketSort",
        "CountSort",
        "EnumerationSort",
        "MergeSort",
        "OddEvenSort",
        "QuickSort",
        "RadixSort",
        "SampleSort",
        "SelectionSort",
    ],
    "Datatype": ["int", "float", "double"],
    "num_procs": [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
    "InputSize": [65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456],
}

# 2A. Modify Metadata Values to Match Grammar

In [4]:
for meta_col, values in META_FIX_DICT.items():
    tk.metadata[meta_col] = tk.metadata[meta_col].replace(values)

# 2B. Filter Metadata Values from Whitelist

In [5]:
print(f"Total files before: {len(tk.profile)}")
tk = tk.filter_metadata(lambda meta: all([meta[key] in META_WHITELIST_DICT[key] for key in META_WHITELIST_DICT.keys()]))
print(f"Total files after: {len(tk.profile)}")

Total files before: 15734
Total files after: 12641


# 2C. Filter Duplicate Metadata Values

Indicates that one profile has incorrect metadata, since all profiles are assumed to be single-trial. Usually from user error (metadata is manually annotated in Adiak).

In [6]:
gb = tk.groupby(["Algorithm", "InputType", "Datatype", "group_num", "InputSize"])
rm_profs = []
for key, ttk in gb.items():
    if ttk.metadata["num_procs"].duplicated().any():
        print(f"Skipping {key} ({len(ttk.profile)} profiles) because it has duplicate num_procs")
        rm_profs += ttk.profile   
tk = tk.filter_profile([p for p in tk.profile if p not in set(rm_profs)])
print(f"Total files after removing duplicates: {len(tk.profile)}")

Skipping ('RadixSort', 'Random', 'double', 2.0, 65536.0) (27 profiles) because it has duplicate num_procs
Skipping ('RadixSort', 'Random', 'double', 2.0, 262144.0) (26 profiles) because it has duplicate num_procs
Total files after removing duplicates: 12588


# 3. Create Features

# 3A. Query the Call Tree

To get performance metrics per node.

In [7]:
nodes = [
    "comp",
    "comp_large",
    "comm",
    "comm_large",
    "comp_small",
    "comm_small"
]
ntk_dict = {n: tk.query(
    th.query.Query().match(
        "*",
        lambda row: row["name"].apply(
            lambda tn: tn == n
        ).all()
    )
) for n in nodes}

# 3B. Compute Features from Performance Data using Queried Thickets

In [8]:
tk = th.Thicket.concat_thickets(
    thickets=list(ntk_dict.values()),
)
# Get mapping of node objects
node_objects = {n.frame["name"]: n for n in [n for n in tk.dataframe.index.get_level_values("node").unique()]}

In [9]:
# Drop duplicate profiles from concat_thickets
# Can't pass these cols in the check or error will be thrown. Won't change the outcome of the check
unhashable_cols = ["libraries", "cmdline"]
tk.metadata = tk.metadata.drop_duplicates(subset=[col for col in tk.metadata.columns if col not in unhashable_cols])

In [10]:
# Nodes not considered in the check. They are only used for their presence T/F
not_considered = ["comp_small", "comm_small"]
profiles_per_node = [set(ntk_dict[n].dataframe.index.get_level_values("profile")) for n in ntk_dict.keys() if n not in not_considered]
# Intersection of the profiles
profile_truth = list(profiles_per_node[0].intersection(*profiles_per_node[1:]))
# Filter the Thicket to only contain these profiles
tk = tk.filter_profile(profile_truth)

In [12]:
metric_cols = [
    "Variance time/rank",
    "Min time/rank",
    "Max time/rank",
    "Avg time/rank",
    "Total time",
]

# Compute metric "Node presence for each node in the performance data"
tk.dataframe["Node presence"] = tk.dataframe["name"].apply(lambda x: False if x is None else True)

In [70]:
# Compute comp/comm
tk.add_root_node(attrs={"name": "comp/comm"})
compcomm = [node for node in tk.graph.traverse() if node.frame["name"] == "comp/comm"][0]
tdf = tk.dataframe.loc[node_objects["comp"], metric_cols].div(tk.dataframe.loc[node_objects["comm"], metric_cols])
for prof in tdf.index:
    tk.dataframe.loc[(compcomm, prof), metric_cols] = tdf.loc[prof]

In [71]:
perf_idx = (
    (
        [
            compcomm, 
            node_objects["comp_large"],
            node_objects["comm_large"]
        ]
    ), metric_cols
)

presence_idx = (
    (
        [
            node_objects["comp_small"],
            node_objects["comm_small"],
        ]
    ), [
        "Node presence"
    ]
)

feature_slices = [perf_idx, presence_idx]

# 3C. Filter Features with NaN Values

In [72]:
print(f"Total profiles before dropping NaNs: {len(tk.profile)}")
for idx in feature_slices:
    any_nan_rows_series = tk.dataframe.loc[idx].isna().apply(lambda x: x.any(), axis=1)
    nan_profs = tk.dataframe.loc[idx][any_nan_rows_series].index.get_level_values("profile").unique()
    tk = tk.filter_profile([p for p in tk.profile if p not in nan_profs])
print(f"Total profiles after dropping NaNs: {len(tk.profile)}")

Total profiles before dropping NaNs: 11348
Total profiles after dropping NaNs: 10628


# 4. Remove Anomalies 

In [ ]:
# Omitting

# 5. Write Model Data

In [73]:
# Shuffle data
tk.metadata = tk.metadata.sample(frac=1.0)

dfs = {}
algs = tk.metadata.reset_index().groupby("Algorithm")
for name, data in algs:
    dfs[name] = pd.DataFrame(data)

dfs = dict(sorted(dfs.items(), key=lambda item: len(item[1]), reverse=True))
for name, data in dfs.items():
    print(f"Algorithm: {name} {sorted(list(data['group_num'].unique()))} has {len(data)} data points")

labels = [
    'MergeSort',
    'SampleSort',
    'OddEvenSort',
    'BitonicSort',
    'RadixSort',
]
num_classes = len(labels)

trim_dict = {}
total_data=0
min_data_points = min([len(dfs[l]) for l in labels])

for label in labels:
    print(f"Label: '{label}' has {len(dfs[label])} data points")
    trim_dict[label] = dfs[label]
    total_data += len(trim_dict[label])
print(f"Total data points: {total_data}")

total_df = pd.concat(trim_dict.values(), axis=0)
total_df = total_df.set_index("profile")

tk = tk.filter_profile(list(total_df.index))

# %% [markdown]
# # 3. Write Thicket to File

tk.feature_slices = feature_slices
tk.node_objects = node_objects

# %%
pickle.dump(tk, open('thicket-modeldata.pkl', 'wb'))

Algorithm: MergeSort [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 11.0, 13.0, 15.0, 16.0, 18.0, 20.0, 21.0, 24.0] has 2322 data points
Algorithm: SampleSort [1.0, 6.0, 9.0, 10.0, 13.0, 14.0, 16.0, 22.0, 25.0] has 2285 data points
Algorithm: OddEvenSort [1.0, 3.0, 5.0, 9.0, 11.0, 13.0, 15.0, 17.0, 18.0, 19.0, 20.0, 24.0] has 2078 data points
Algorithm: BitonicSort [1.0, 4.0, 5.0, 6.0, 7.0, 10.0, 11.0, 13.0, 15.0, 16.0, 23.0] has 1761 data points
Algorithm: QuickSort [3.0, 4.0, 5.0, 11.0, 17.0, 18.0] has 643 data points
Algorithm: RadixSort [7.0, 9.0, 10.0, 23.0] has 591 data points
Algorithm: EnumerationSort [15.0, 20.0] has 286 data points
Algorithm: BucketSort [19.0] has 268 data points
Algorithm: CountSort [19.0] has 266 data points
Algorithm: SelectionSort [3.0, 16.0] has 128 data points
Label: 'MergeSort' has 2322 data points
Label: 'SampleSort' has 2285 data points
Label: 'OddEvenSort' has 2078 data points
Label: 'BitonicSort' has 1761 data points
Label: 'RadixSort' has 591 data points
Tot